In [1]:
from concurrent import futures

class Solution:
    def crawl(self, startUrl: str, htmlParser: 'HtmlParser'):
        hostname = lambda url: url.split('/')[2]
        seen = {startUrl}
    
        with futures.ThreadPoolExecutor(max_workers=16) as executor:
            tasks = deque([executor.submit(htmlParser.getUrls, startUrl)])
            while tasks:
                for url in tasks.popleft().result():
                    if url not in seen and hostname(startUrl) == hostname(url):
                        seen.add(url)
                        tasks.append(executor.submit(htmlParser.getUrls, url))
        
        return list(seen)

In [3]:
import threading
from queue import Queue

class Solution:
    def crawl(self, startUrl: str, htmlParser: 'HtmlParser'):
        def hostname(url):
            start = len("http://")
            i = start
            while i < len(url) and url[i] != "/":
                i += 1
            return url[start:i]
        
        queue = Queue()
        seen = {startUrl}
        start_hostname = hostname(startUrl)
        seen_lock = threading.Lock()
        
        def worker():
            while True:
                url = queue.get()
                if url is None:
                    return

                for next_url in htmlParser.getUrls(url):
                    if next_url not in seen and hostname(next_url) == start_hostname:
                        seen_lock.acquire()
                        # Acquire lock to ensure urls are no enqueed multiple times
                        if next_url not in seen:
                            seen.add(next_url)
                            queue.put(next_url)
                        seen_lock.release()
                queue.task_done()
        
        num_workers = 8
        workers = []
        queue.put(startUrl)
        
        for i in range(num_workers):
            t = threading.Thread(target=worker)
            t.start()
            workers.append(t)
        
        # Wait until empty
        queue.join()
        
        for i in range(num_workers):
            queue.put(None)
        for t in workers:
            t.join()
        
        return list(seen)

In [5]:
from concurrent import futures
from queue import Queue

import threading
class Solution(object):
    def crawl(self, startUrl, htmlParser):
        common_host = startUrl.split('/')[2]
        queue, seen, seen_lock = Queue(), {startUrl}, threading.Lock()
        def worker():
            try:
                while True:
                    url = queue.get(timeout=0.015) # make sure wait up to the time required for the `getUrls` call
                    for next_url in htmlParser.getUrls(url):
                        if next_url not in seen and next_url.split('/')[2] == common_host:
                            seen_lock.acquire()
                            # Acquire lock to ensure urls are no enqueed multiple times
                            if next_url not in seen:
                                seen.add(next_url)
                                queue.put(next_url)
                            seen_lock.release()
                    queue.task_done()
            except: pass
        
        queue.put(startUrl)
        with futures.ThreadPoolExecutor(max_workers=8) as executor:
            for i in range(8): executor.submit(worker)
        
        return list(seen)